In [1]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

# .env 파일 로드
load_dotenv("../.env")

# 환경 변수에서 Azure OpenAI 설정 로드
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("OPENAI_API_VERSION")

# 환경변수 로드 상태 확인
print("=== Azure OpenAI 환경변수 로드 상태 ===")
print(f"🔗 Endpoint: {endpoint}")
print(f"🚀 Deployment: {deployment}")
print(f"🔑 API Key: {'✅ 로드됨' if api_key else '❌ 없음'}")
print(f"📅 API Version: {api_version}")
print()

# 필수 환경변수 검증
missing_vars = []
if not endpoint:
    missing_vars.append("AZURE_OPENAI_ENDPOINT")
if not deployment:
    missing_vars.append("AZURE_OPENAI_CHAT_DEPLOYMENT")
if not api_key:
    missing_vars.append("AZURE_OPENAI_API_KEY")
if not api_version:
    missing_vars.append("OPENAI_API_VERSION")

if missing_vars:
    print(f"❌ 누락된 환경변수: {', '.join(missing_vars)}")
    print("⚠️  .env 파일을 확인해주세요.")
else:
    print("✅ 모든 필수 환경변수가 로드되었습니다.")
print("="*50)

=== Azure OpenAI 환경변수 로드 상태 ===
🔗 Endpoint: https://hspar-m7k2pfor-swedencentral.openai.azure.com/
🚀 Deployment: gpt-4o-mini
🔑 API Key: ✅ 로드됨
📅 API Version: 2024-06-01

✅ 모든 필수 환경변수가 로드되었습니다.


In [2]:
# Azure OpenAI 클라이언트 생성
try:
    client = AzureOpenAI(
        api_version=api_version,
        azure_endpoint=endpoint,
        api_key=api_key,
    )
    print("✅ Azure OpenAI 클라이언트가 성공적으로 생성되었습니다.")
except Exception as e:
    print(f"❌ 클라이언트 생성 실패: {e}")
    raise

# 배포된 모델 목록 확인 시도
print("\n🔍 Azure OpenAI 배포 상태 진단...")
try:
    # REST API를 통한 배포 목록 확인
    import requests
    import json
    
    headers = {
        "api-key": api_key,
        "Content-Type": "application/json"
    }
    
    # 배포 목록 조회 URL
    deployments_url = f"{endpoint.rstrip('/')}/openai/deployments?api-version={api_version}"
    
    print(f"📡 요청 URL: {deployments_url}")
    
    response = requests.get(deployments_url, headers=headers, timeout=10)
    
    if response.status_code == 200:
        deployments = response.json()
        print("\n📋 현재 배포된 모델들:")
        
        if "data" in deployments and deployments["data"]:
            for deployment_info in deployments["data"]:
                deployment_id = deployment_info.get("id", "N/A")
                model = deployment_info.get("model", "N/A")
                status = deployment_info.get("status", "N/A")
                print(f"   📌 배포명: {deployment_id}")
                print(f"      모델: {model}")
                print(f"      상태: {status}")
                print()
        else:
            print("   ⚠️  배포된 모델이 없습니다.")
    else:
        print(f"   ❌ 배포 목록 조회 실패: {response.status_code} - {response.text}")
        
except Exception as e:
    print(f"   ⚠️  배포 상태 확인 중 오류: {e}")
    print("   💡 수동으로 Azure Portal에서 배포 상태를 확인해주세요.")

print("\n" + "="*50)

✅ Azure OpenAI 클라이언트가 성공적으로 생성되었습니다.

🔍 Azure OpenAI 배포 상태 진단...
📡 요청 URL: https://hspar-m7k2pfor-swedencentral.openai.azure.com/openai/deployments?api-version=2024-06-01
   ❌ 배포 목록 조회 실패: 404 - {"error":{"code":"404","message": "Resource not found"}}

   ❌ 배포 목록 조회 실패: 404 - {"error":{"code":"404","message": "Resource not found"}}



In [3]:
# Azure OpenAI API 연결 고급 진단
print("🔍 Azure OpenAI API 고급 진단 시작...")
print()
# https://hspar-m7k2pfor-swedencentral.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview
# 다양한 API 버전 시도
api_versions_to_try = [
    "2025-01-01-preview",
    "2024-07-18-preview",
    "2024-06-01", 
    "2024-02-15-preview",
    "2023-12-01-preview",
    "2023-05-15"
]

success = False
working_api_version = None

for test_version in api_versions_to_try:
    print(f"📡 API 버전 {test_version} 테스트 중...")
    
    try:
        # 기본 연결 테스트 (models 엔드포인트)
        import requests
        headers = {
            "api-key": api_key,
            "Content-Type": "application/json"
        }
        
        # 모델 목록 조회 시도
        models_url = f"{endpoint.rstrip('/')}/openai/models?api-version={test_version}"
        response = requests.get(models_url, headers=headers, timeout=10)
        
        if response.status_code == 200:
            print(f"   ✅ API 버전 {test_version} 작동 확인!")
            working_api_version = test_version
            
            # 사용 가능한 모델 리스트 표시
            models_data = response.json()
            if "data" in models_data:
                print(f"   📋 사용 가능한 모델들:")
                for model in models_data["data"][:5]:  # 처음 5개만 표시
                    print(f"      - {model.get('id', 'Unknown')}")
                if len(models_data["data"]) > 5:
                    print(f"      ... 및 {len(models_data['data']) - 5}개 추가 모델")
            
            success = True
            break
        else:
            print(f"   ❌ {response.status_code}: {response.text[:100]}")
            
    except Exception as e:
        print(f"   ❌ 오류: {str(e)[:100]}")
    
    print()

if success and working_api_version:
    print(f"🎉 작동하는 API 버전 발견: {working_api_version}")
    print()
    
    # 작동하는 API 버전으로 체팅 모델 테스트
    print(f"🤖 {working_api_version}로 체팅 모델 테스트 시도...")
    
    try:
        # 새로운 클라이언트 생성
        working_client = AzureOpenAI(
            api_version=working_api_version,
            azure_endpoint=endpoint,
            api_key=api_key,
        )
        
        # 간단한 체팅 요청
        response = working_client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant. Please respond briefly."
                },
                {
                    "role": "user",
                    "content": "Hello! Please say 'Connection test successful' in Korean."
                }
            ],
            max_tokens=100,
            temperature=0.7,
            model=deployment
        )
        
        # 응답 출력
        response_content = response.choices[0].message.content
        print(f"✅ API 연결 테스트 성공!")
        print(f"🤖 AI 응답: {response_content}")
        print()
        
        # 토큰 사용량 정보
        if hasattr(response, 'usage'):
            print("📊 토큰 사용량:")
            print(f"   - 입력 토큰: {response.usage.prompt_tokens}")
            print(f"   - 출력 토큰: {response.usage.completion_tokens}")
            print(f"   - 총 토큰: {response.usage.total_tokens}")
        
        print(f"\n💡 권장 사항: .env 파일의 OPENAI_API_VERSION을 '{working_api_version}'로 업데이트하세요.")
        
    except Exception as e:
        print(f"❌ 체팅 모델 테스트 실패: {e}")
        print("\n🔍 추가 진단 정보:")
        print(f"   - 현재 배포명: {deployment}")
        print(f"   - 엔드포인트: {endpoint}")
        print("   - Azure Portal에서 해당 리소스와 배포 상태를 확인해주세요.")
else:
    print("❌ 모든 API 버전에서 오류 발생")
    print("\n🚑 가능한 원인:")
    print("1. Azure OpenAI 리소스가 삭제되었거나 비활성화됨")
    print("2. 엔드포인트 URL이 잘못됨 (지역 또는 리소스명 오류)")
    print("3. API 키가 만료되었거나 권한이 없음")
    print("4. 네트워크 연결 문제 또는 방화벽 차단")
    print("\n📝 해결 방법:")
    print("- Azure Portal에서 리소스 상태 확인")
    print("- 엔드포인트 URL 재확인")
    print("- API 키 재생성")

print("\n" + "="*50)

🔍 Azure OpenAI API 고급 진단 시작...

📡 API 버전 2025-01-01-preview 테스트 중...
   ✅ API 버전 2025-01-01-preview 작동 확인!
   📋 사용 가능한 모델들:
      - dall-e-3-3.0
      - dall-e-2-2.0
      - tts-001
      - tts-hd-001
      - whisper-001
      ... 및 202개 추가 모델
🎉 작동하는 API 버전 발견: 2025-01-01-preview

🤖 2025-01-01-preview로 체팅 모델 테스트 시도...
   ✅ API 버전 2025-01-01-preview 작동 확인!
   📋 사용 가능한 모델들:
      - dall-e-3-3.0
      - dall-e-2-2.0
      - tts-001
      - tts-hd-001
      - whisper-001
      ... 및 202개 추가 모델
🎉 작동하는 API 버전 발견: 2025-01-01-preview

🤖 2025-01-01-preview로 체팅 모델 테스트 시도...
✅ API 연결 테스트 성공!
🤖 AI 응답: Connection test successful in Korean is: 연결 테스트 성공.

📊 토큰 사용량:
   - 입력 토큰: 33
   - 출력 토큰: 12
   - 총 토큰: 45

💡 권장 사항: .env 파일의 OPENAI_API_VERSION을 '2025-01-01-preview'로 업데이트하세요.

✅ API 연결 테스트 성공!
🤖 AI 응답: Connection test successful in Korean is: 연결 테스트 성공.

📊 토큰 사용량:
   - 입력 토큰: 33
   - 출력 토큰: 12
   - 총 토큰: 45

💡 권장 사항: .env 파일의 OPENAI_API_VERSION을 '2025-01-01-preview'로 업데이트하세요.



In [4]:
# 임베딩 모델 테스트 (선택사항)
print("\n🔄 임베딩 모델 연결 테스트...")

# 임베딩 배포 정보 확인
embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
print(f"🎯 임베딩 배포: {embedding_deployment}")

if embedding_deployment:
    try:
        # 임베딩 테스트
        embedding_response = client.embeddings.create(
            input="This is a test sentence for embedding.",
            model=embedding_deployment
        )
        
        embedding_vector = embedding_response.data[0].embedding
        print(f"✅ 임베딩 모델 연결 성공!")
        print(f"📐 임베딩 벡터 차원: {len(embedding_vector)}")
        print(f"🔢 벡터 샘플 (처음 5개): {embedding_vector[:5]}")
        
    except Exception as e:
        print(f"❌ 임베딩 모델 연결 실패: {e}")
else:
    print("⚠️  임베딩 배포 정보가 .env 파일에 없습니다.")

print("\n" + "="*50)
print("🎉 Azure OpenAI 연결 테스트 완료!")


🔄 임베딩 모델 연결 테스트...
🎯 임베딩 배포: text-embedding-ada-002
✅ 임베딩 모델 연결 성공!
📐 임베딩 벡터 차원: 1536
🔢 벡터 샘플 (처음 5개): [-0.014208625070750713, 0.004100165795534849, 0.0012239423813298345, 0.002518861787393689, -0.0011529652401804924]

🎉 Azure OpenAI 연결 테스트 완료!
✅ 임베딩 모델 연결 성공!
📐 임베딩 벡터 차원: 1536
🔢 벡터 샘플 (처음 5개): [-0.014208625070750713, 0.004100165795534849, 0.0012239423813298345, 0.002518861787393689, -0.0011529652401804924]

🎉 Azure OpenAI 연결 테스트 완료!


In [5]:
# 최종 테스트: 수정된 .env 설정으로 연결 테스트
print("🔄 수정된 .env 설정으로 최종 테스트...")
print()

# .env 파일 재로드
from dotenv import load_dotenv
import os

# 기존 환경변수 청소 후 재로드
for key in ['AZURE_OPENAI_ENDPOINT', 'AZURE_OPENAI_CHAT_DEPLOYMENT', 'AZURE_OPENAI_API_KEY', 'OPENAI_API_VERSION']:
    if key in os.environ:
        del os.environ[key]

# .env 파일 재로드
load_dotenv("../.env", override=True)

# 새로운 환경 변수 확인
print("🔄 새로운 환경변수:")
print(f"   API Version: {os.getenv('OPENAI_API_VERSION')}")
print(f"   Deployment: {os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT')}")
print()

# 새로운 설정으로 클라이언트 생성 및 테스트
try:
    from openai import AzureOpenAI
    
    final_client = AzureOpenAI(
        api_version=os.getenv('OPENAI_API_VERSION'),
        azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
        api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    )
    
    # 최종 연결 테스트
    response = final_client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant. Please respond briefly."
            },
            {
                "role": "user",
                "content": "Hello! Please say 'Final test successful' in Korean."
            }
        ],
        max_tokens=100,
        temperature=0.7,
        model=os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT')
    )
    
    response_content = response.choices[0].message.content
    print("✅ 최종 API 연결 테스트 성공!")
    print(f"🤖 AI 응답: {response_content}")
    print()
    
    if hasattr(response, 'usage'):
        print("📊 토큰 사용량:")
        print(f"   - 입력 토큰: {response.usage.prompt_tokens}")
        print(f"   - 출력 토큰: {response.usage.completion_tokens}")
        print(f"   - 총 토큰: {response.usage.total_tokens}")
    
    print("\n🎉 모든 테스트 완료! Azure OpenAI 연결이 정상적으로 작동합니다.")
    
except Exception as e:
    print(f"❌ 최종 테스트 실패: {e}")
    
print("\n" + "="*60)

🔄 수정된 .env 설정으로 최종 테스트...

🔄 새로운 환경변수:
   API Version: 2024-06-01
   Deployment: gpt-4o-mini

✅ 최종 API 연결 테스트 성공!
🤖 AI 응답: Final test successful in Korean is: 최종 테스트 성공.

📊 토큰 사용량:
   - 입력 토큰: 33
   - 출력 토큰: 13
   - 총 토큰: 46

🎉 모든 테스트 완료! Azure OpenAI 연결이 정상적으로 작동합니다.

✅ 최종 API 연결 테스트 성공!
🤖 AI 응답: Final test successful in Korean is: 최종 테스트 성공.

📊 토큰 사용량:
   - 입력 토큰: 33
   - 출력 토큰: 13
   - 총 토큰: 46

🎉 모든 테스트 완료! Azure OpenAI 연결이 정상적으로 작동합니다.

